<a href="https://colab.research.google.com/github/pxuanbach/recommendation-system/blob/main/Demo_model_based_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo model-based approach

### 1. Cài đặt và thêm các package cần thiết

In [3]:
!pip install pyspark
!pip install pandas

You should consider upgrading via the 'c:\users\back\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


  Using cached pandas-1.5.2-cp39-cp39-win_amd64.whl (10.9 MB)
  Using cached numpy-1.24.0-cp39-cp39-win_amd64.whl (14.9 MB)
  Using cached pytz-2022.7-py2.py3-none-any.whl (499 kB)


You should consider upgrading via the 'c:\users\back\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [4]:
import pandas as pd
from pyspark.sql.functions import col, explode
from pyspark import SparkContext

Tạo spark session

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Recommendations Demo').getOrCreate()

### 2. Tải dữ liệu lên

In [6]:
movies = spark.read.csv('movies.csv', header=True)
ratings = spark.read.csv('ratings.csv', header=True)

In [7]:
ratings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



Chỉnh sửa schema

In [8]:
ratings = ratings.\
    withColumn('userId', col('userId').cast('integer')).\
    withColumn('movieId', col('movieId').cast('integer')).\
    withColumn('rating', col('rating').cast('float')).\
    drop('timestamp')
ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    231|   5.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    349|   4.0|
+------+-------+------+
only showing top 20 rows



### 3. Tính toán độ thưa thớt

In [9]:
# Count the total number of ratings in the dataset
numerator = ratings.select("rating").count()
print("Total number of ratings", numerator)

# Count the number of distinct userIds and distinct movieIds
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()
print("Number users", num_users, "Number movies", num_movies)

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_movies

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

Total number of ratings 40022
Number users 274 Number movies 6222
The ratings dataframe is  97.65% empty.


### 4. Nhóm dữ liệu bằng cách tính tổng số lượt ratings

Nhóm user

In [10]:
# Group data by userId, count ratings
userId_ratings = ratings.groupBy("userId").count().orderBy('count', ascending=False)
userId_ratings.show()

+------+-----+
|userId|count|
+------+-----+
|    68| 1260|
|   249| 1046|
|   182|  977|
|   177|  904|
|   232|  862|
|   274|  793|
|   105|  722|
|    19|  703|
|   111|  646|
|   217|  613|
|   140|  608|
|    91|  575|
|    28|  570|
|   219|  528|
|    89|  518|
|    64|  517|
|   226|  507|
|    18|  502|
|    57|  476|
|    21|  443|
+------+-----+
only showing top 20 rows



Nhóm movie

In [11]:
# Group data by movieId, count ratings
movieId_ratings = ratings.groupBy("movieId").count().orderBy('count', ascending=False)
movieId_ratings.show()

+-------+-----+
|movieId|count|
+-------+-----+
|    296|  144|
|    356|  144|
|    318|  140|
|   2571|  129|
|    593|  125|
|    260|  113|
|    110|  104|
|    480|  103|
|   1196|  102|
|    589|  100|
|      1|   99|
|   1210|   96|
|   1198|   96|
|    780|   95|
|     47|   95|
|    150|   94|
|   2858|   90|
|    527|   90|
|    592|   89|
|   2028|   89|
+-------+-----+
only showing top 20 rows



### 5. Xây dựng mô hình ALS

In [12]:
from pyspark.ml.recommendation import ALS

In [13]:
# Create test and train set
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 1234)

# Create ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

# Confirm that a model called "als" was created
type(als)

pyspark.ml.recommendation.ALS

Điều chỉnh siêu tham số cho ALS model
- rank (rank of the factorization) hạng của thừa số hóa.
- regParam (regularization parameter (>= 0)) là tham số chính quy hóa.

In [14]:
# Import the requisite items
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100]) \
            .addGrid(als.regParam, [.01, .05, .1]) \
            .build()

Sau đó tạo RegressionEvaluator

In [15]:
# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(
           metricName="rmse", 
           labelCol="rating", 
           predictionCol="prediction") 
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  9


In [16]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

### 6. Kiểm tra các thông số được lựa chọn cho mô hình tốt nhất

In [17]:
#Fit cross validator to the 'train' dataset
model = cv.fit(train)

#Extract best model from the cv model above
best_model = model.bestModel

In [18]:
print("**Best Model**")
# Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())
# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())
# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

**Best Model**
  Rank: 10
  MaxIter: 10
  RegParam: 0.1


Xem thử dự đoán

In [19]:
# View the predictions
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

0.9496723878260326


In [20]:
test_predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   148|    356|   4.0|  3.854917|
|   148|   4896|   4.0| 3.3067513|
|   148|   4993|   3.0|  3.066208|
|   148|   7153|   3.0| 3.6244917|
|   148|   8368|   4.0| 3.7092826|
|   148|  40629|   5.0| 2.7161942|
|   148|  50872|   3.0| 3.0652325|
|   148|  60069|   4.5| 3.7623978|
|   148|  69757|   3.5| 3.7207644|
|   148|  72998|   4.0| 3.1236095|
|   148|  81847|   4.5|  3.531046|
|   148|  98491|   5.0| 3.5566206|
|   148| 115617|   3.5|  4.017053|
|   148| 122886|   3.5| 3.8015947|
|   243|     62|   5.0| 4.0809207|
|   243|    145|   4.0| 4.7360873|
|   243|    248|   4.0|  4.787063|
|   243|    520|   5.0| 3.8411067|
|   243|    589|   4.0| 4.7180047|
|   243|    592|   3.0|  4.171158|
+------+-------+------+----------+
only showing top 20 rows



### 7. Đưa ra đề xuất

In [21]:
# Generate n Recommendations for all users
nrecommendations = best_model.recommendForAllUsers(10)
nrecommendations.limit(10).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{1658, 6.2070546...|
|     2|[{1658, 5.0419745...|
|     3|[{70946, 4.968466...|
|     4|[{720, 5.5889835}...|
|     5|[{213, 5.071801},...|
|     6|[{150548, 5.47501...|
|     7|[{47423, 4.947241...|
|     8|[{28, 5.1227274},...|
|     9|[{1658, 5.741402}...|
|    10|[{71579, 4.868856...|
+------+--------------------+



In [22]:
nrecommendations = nrecommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', col("rec_exp.movieId"), col("rec_exp.rating"))

nrecommendations.limit(10).show()

+------+-------+---------+
|userId|movieId|   rating|
+------+-------+---------+
|     1|   1658|6.2070546|
|     1|    898|5.6630154|
|     1|   1250| 5.568542|
|     1|   1262| 5.562644|
|     1|   7842| 5.527656|
|     1|  44195|5.4760265|
|     1|  55118| 5.418627|
|     1|   3275| 5.413618|
|     1|   5466| 5.406734|
|     1|   1408| 5.396342|
+------+-------+---------+



### 8. Các đề xuất có hợp lý không?

In [23]:
nrecommendations.join(movies, on='movieId').filter('userId = 100').show()

+-------+------+---------+--------------------+--------------------+
|movieId|userId|   rating|               title|              genres|
+-------+------+---------+--------------------+--------------------+
| 122886|   100|5.2960615|Star Wars: Episod...|Action|Adventure|...|
|   1262|   100| 5.058984|Great Escape, The...|Action|Adventure|...|
|  69134|   100|  4.97764|   Antichrist (2009)|       Drama|Fantasy|
|   3200|   100| 4.960183|Last Detail, The ...|        Comedy|Drama|
| 147330|   100|4.9516926|Sherlock Holmes a...|               Crime|
| 104780|   100|4.9516926|Mystery of the Th...|Adventure|Animati...|
| 173351|   100|4.9516926|Wow! A Talking Fi...|Animation|Childre...|
| 159811|   100|4.9516926|The Bremen Town M...|Animation|Drama|F...|
| 142020|   100|4.9516926|        Oscar (1967)|              Comedy|
|   2131|   100|4.9516926|Autumn Sonata (Hö...|               Drama|
+-------+------+---------+--------------------+--------------------+



In [34]:
ratings.join(movies, on='movieId').filter('userId = 100').sort('rating', ascending=False).show()

+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|   2423|   100|   5.0|Christmas Vacatio...|              Comedy|
|   1101|   100|   5.0|      Top Gun (1986)|      Action|Romance|
|   4041|   100|   5.0|Officer and a Gen...|       Drama|Romance|
|   1958|   100|   5.0|Terms of Endearme...|        Comedy|Drama|
|   5620|   100|   5.0|Sweet Home Alabam...|      Comedy|Romance|
|    919|   100|   4.5|Wizard of Oz, The...|Adventure|Childre...|
|    934|   100|   4.5|Father of the Bri...|              Comedy|
|     28|   100|   4.5|   Persuasion (1995)|       Drama|Romance|
|     95|   100|   4.5| Broken Arrow (1996)|Action|Adventure|...|
|   1028|   100|   4.5| Mary Poppins (1964)|Children|Comedy|F...|
|   1091|   100|   4.5|Weekend at Bernie...|              Comedy|
|     16|   100|   4.5|       Casino (1995)|         Crime|Drama|
|   1246| 

### 9. Lưu model

In [47]:
save_model_path = "trained"
best_model.save(path=save_model_path)

Py4JJavaError: An error occurred while calling o13708.save.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.ja